# Regularization

Let's improve our understanding of what impacted **Titanic** passengers' chance of survival
- We will use logistic classifiers which are easy to interpret
- Remember we already did it with statsmodels in lecture "Decision Science - Logistic Regression"
- We were using `p-values` & statistical assumptions to detect which features were irrelevant / don't generalize
- This time, we will use `regularization` to detect relevant/irrelevant features based on under/overfitting criteria
- **Our goal is to compare `L1` and `L2` penalties**

## 1. We load and preprocess the data for you

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/Machine%20Learning%20Datasets/ML_titanic_dataset_encoded.csv")

# the dataset is already one-hot-encoded
data.head()

,survived,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,0,3,22.0,1,0,7.2500,0,0,1,0,0,0,1
1,1,1,38.0,1,0,71.2833,1,1,0,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,1,0,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,0,1,0,0,0,1


In [3]:
# We build X and y

y = data["survived"]
X = data.drop(columns="survived")
X.head()

,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,3,22.0,1,0,7.2500,0,0,1,0,0,0,1
1,1,38.0,1,0,71.2833,1,1,0,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,1,0,0,0,0,1
4,3,35.0,0,0,8.0500,0,0,1,0,0,0,1


In [6]:
# We MinMaxScale our features for you
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(X)
X_scaled = scaler.transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X.shape
X_scaled.head()

,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,1.0,0.271174,0.2,0.0,0.014151,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0,0.472229,0.2,0.0,0.139136,1.0,1.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.321438,0.0,0.0,0.015469,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,0.434531,0.2,0.0,0.103644,1.0,1.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.434531,0.0,0.0,0.015713,0.0,0.0,1.0,0.0,0.0,0.0,1.0


## 1.  Logistic Regression without regularization

❓ Rank the features by decreasing order of importance after training a simple **non-regularized** Logistic Regression (i.e. look at the coefficients after fitting)
- Careful: `LogisticRegression` is penalized by default
  - take a look at the [penalty parameter](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) to find out how to remove the penalty)
- Increase `max_iter` to a larger number until model converges

<details>
    <summary>Hint</summary>
    <img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/05-ML/05-Model-Tuning/model_selection.png" alt="penalizing a regression" width="500" height="600">
</details>

In [7]:
from sklearn.linear_model import LogisticRegression

In [31]:
# YOUR CODE HERE
model = LogisticRegression(penalty='none', max_iter=5000)
model.fit(X, y)
coef = model.coef_
feature_names = X.columns
feature_names
coef
coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coef[0]})
coefficients_df 
coefficients_df_sorted = coefficients_df.sort_values(by='Coefficient', ascending=False)
coefficients_df_sorted 


/Users/Laetitia/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1183: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


,Feature,Coefficient
5,sex_female,2.670565
6,class_First,2.330395
8,who_child,1.327026
0,pclass,1.244988
4,fare,0.002652
1,age,-0.027688
3,parch,-0.146846
2,sibsp,-0.495353
7,class_Third,-2.430144
9,embark_town_Cherbourg,-5.513905


❓How do you interpret, in plain English, the value for the coefficient `sex_female`?

<details>
    <summary>Answer</summary>

> "All other things being equal (such as age, ticket class etc...),
being a women increases your log-odds of survival by 2.67 (your coef value)"
    
> "Controling for all other explaining factors available in this dataset,
being a women increases your odds of survival by exp(2.67) = 14"

</details>


❓ What is the feature that most impacts the chances of survival according to your model?  
Fill the `top_1_feature` list below with the name of this feature

In [34]:
top_1_feature = ["embark_town_Queenstown"]

In [35]:
from nbresult import ChallengeResult
result = ChallengeResult('unregularized', top_1_feature = top_1_feature)
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/Laetitia/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/Laetitia/code/juliensoudet/05-ML/05-Model-Tuning/data-regularization/tests
plugins: asyncio-0.19.0, typeguard-2.13.3, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_unregularized.py::TestUnregularized::test_top_1 PASSED              [100%]

============================== 1 passed in 0.04s ===============================


💯 You can commit your code:

git add tests/unregularized.pickle

git commit -m 'Completed unregularized step'

git push origin master



In [36]:
! git add tests/unregularized.pickle

! git commit -m 'Completed unregularized step'

! git push origin master

[master 02f441e] Completed unregularized step
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 tests/unregularized.pickle
Enumerating objects: 15, done.
Counting objects: 100% (15/15), done.
Delta compression using up to 4 threads
Compressing objects: 100% (14/14), done.
Writing objects: 100% (15/15), 4.30 KiB | 1.07 MiB/s, done.
Total 15 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
remote: Resolving deltas: 100% (3/3), done.
To github.com:juliensoudet/data-regularization.git
 * [new branch]      master -> master


## 2.  Logistic Regression with an L2 penalty

Let's use a **Logistic model** whose log-loss has been penalized with a **L2** term to figure out the **most important features** without overfitting.  
This is the "classification" equivalent to the "Ridge" regressor

❓ Instantiate a **strongly regularized** `LogisticRegression` and rank its features by importance (look at the coefficients)
- By "strongly regularized" we mean "more than Sklearn's default regularization factor". 
- Sklearn's default values are very useful orders of magnitudes to keep in mind for "scaled features"

In [37]:
# YOUR CODE HERE
model = LogisticRegression(penalty='l2', max_iter=1000)
model.fit(X, y)

LogisticRegression(max_iter=1000)

In [38]:
coef = model.coef_
feature_names = X.columns
feature_names
coef
coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coef[0]})
coefficients_df 
coefficients_df_sorted = coefficients_df.sort_values(by='Coefficient', ascending=False)
coefficients_df_sorted 


,Feature,Coefficient
5,sex_female,2.532460
8,who_child,1.106338
6,class_First,0.325554
9,embark_town_Cherbourg,0.208946
4,fare,0.002663
1,age,-0.028878
11,embark_town_Southampton,-0.093714
3,parch,-0.122466
10,embark_town_Queenstown,-0.350159
7,class_Third,-0.434173


❓ What are the top 2 features driving chances of survival according to your model?  
Fill the `top_2_features` list below with the name of these features

In [39]:
top_2_features = ["sex_female", "who_child"]

#### 🧪 Test your code below

In [40]:
from nbresult import ChallengeResult
result = ChallengeResult('ridge', top_2 = top_2_features)
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/Laetitia/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/Laetitia/code/juliensoudet/05-ML/05-Model-Tuning/data-regularization/tests
plugins: asyncio-0.19.0, typeguard-2.13.3, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_ridge.py::TestRidge::test_top2 PASSED                               [100%]

============================== 1 passed in 0.03s ===============================


💯 You can commit your code:

git add tests/ridge.pickle

git commit -m 'Completed ridge step'

git push origin master



In [41]:
! git add tests/ridge.pickle

! git commit -m 'Completed ridge step'

! git push origin master

[master 072b98d] Completed ridge step
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 tests/ridge.pickle
Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 4 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 437 bytes | 437.00 KiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:juliensoudet/data-regularization.git
   02f441e..072b98d  master -> master


## 2. Logistic Regression with an L1 penalty

This time, we'll use a logistic model whose log-loss has been penalized with a **L1** term to **filter-out the less important features**.  
This is the "classification" equivalent to the **Lasso** regressor

❓ Instantiate a **strongly regularized** `LogisticRegression` and rank its features by importance

In [44]:
# YOUR CODE HERE
model = LogisticRegression(penalty='l1', solver = "liblinear", max_iter=1000)
model.fit(X, y)
coef = model.coef_
feature_names = X.columns
feature_names
coef
coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coef[0]})
coefficients_df 
coefficients_df_sorted = coefficients_df.sort_values(by='Coefficient', ascending=False)
coefficients_df_sorted 



,Feature,Coefficient
5,sex_female,2.589517
8,who_child,1.174559
6,class_First,0.843549
9,embark_town_Cherbourg,0.264871
4,fare,0.003066
11,embark_town_Southampton,0.000000
1,age,-0.026413
10,embark_town_Queenstown,-0.107490
3,parch,-0.125959
0,pclass,-0.183235


❓ What are the features that have absolutely no impact on chances of survival, according to your L1 model?  
Fill the `zero_impact_features` list below with the name of these features; you may have to add elements to the list.

- Do you notice how some of them were "highly important" according to the non-regularized model? 
- From now on, we will always regularize our linear models!

In [ ]:
zero_impact_features = ["fare", "embark_town_Southampton", "", ""]

#### 🧪 Test your code below

In [ ]:
from nbresult import ChallengeResult
result = ChallengeResult('lasso', zero_impact_features = zero_impact_features)
result.write()
print(result.check())

**🏁 Congratulation! Don't forget to commit and push your notebook**